In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
from math import sqrt

In [2]:
empresas=pd.read_excel('data_final.xlsx')

In [3]:
symbols=empresas['symbol']

In [4]:
class portfolio:
    def __init__(self,symbols):
        self.symbs=sorted(symbols)
    @property
    def Close(self):
        data=yf.download(tickers=self.symbs, start='2019-6-1',progress=0).Close
        data=data.dropna(axis=0)
        return data   
    @property
    def matriz_rendimientos(self):
        def rendimiento(comp): return np.diff(np.log(comp)) # return [log(comp[i]/comp[i-1]) for i in range(1,len(comp))]
        df_rend=self.Close.apply(rendimiento,axis=0)
        return df_rend
    @property
    def rend_volt_anual(self):
        rendimientos=self.matriz_rendimientos
        medias=list(rendimientos.mean()*252)
        varianzas=list(rendimientos.std()*sqrt(len(rendimientos)*252/(len(rendimientos)+1)))
        return medias,varianzas
    @property
    def MVC(self):
        medias,varianzas=self.rend_volt_anual
        Matriz_Si=[[0]*len(varianzas) for i in range(len(varianzas))]
        for i,s_i in zip(range(len(Matriz_Si)),range(len(varianzas))):
                Matriz_Si[i][s_i]=varianzas[s_i]
        Matriz_Si=np.array(Matriz_Si)
        Matrizcorr=self.Close.corr()
        Matrizcorr=np.array(Matrizcorr)
        return Matriz_Si@Matrizcorr@Matriz_Si.T
    @property
    def portafolios(self):
        mvc=self.MVC
        Eri=self.rend_volt_anual[0]
        def sigmaW(Port): return sqrt(Port@mvc@Port.T)
        def rendW(Porc_w): return sum([p*r for p,r in zip(Eri,Porc_w)])
        portfolios=np.random.random((10_000,len(self.symbs)))
        porcentajes=[x/sum(x) for x in portfolios]
        y=[sigmaW(x) for x in porcentajes]
        b,m,a=np.quantile(y,(0.25,0.5,0.75))
        porcentajes_portafolios={}
        def clustering_portfolio(p):
            return "b" if p<=b else "m" if p<=m else "a"        
        clasif=list(map(clustering_portfolio,y))
        rendimiento=list(map(rendW,porcentajes))
        return pd.DataFrame([(p,c,r,v) for p,c,r,v in 
                        zip(porcentajes,clasif,rendimiento,y)],
        columns=["Portafolio",'Clasificación','Rendimiento','Volatilidad'])                            

In [5]:
my_portfolio=portfolio(list(symbols[:20]))

In [6]:
df_portafolios=my_portfolio.portafolios

In [7]:
df_portafolios[df_portafolios['Clasificación']=='b'].sort_values('Volatilidad')

,Portafolio,Clasificación,Rendimiento,Volatilidad
4478,"[0.05472645784440923, 0.016949711956745234, 0....",b,0.107349,0.207316
6907,"[0.03604228283751039, 0.04955863780425354, 0.0...",b,0.120088,0.214278
970,"[0.023413804693322837, 0.018677584232410747, 0...",b,0.103054,0.214447
6066,"[0.009285428409306025, 0.10464521329641943, 0....",b,0.098106,0.215310
4670,"[0.100521757380107, 0.0013093909520491775, 0.0...",b,0.113900,0.216121
...,...,...,...,...
9892,"[0.038421335935235355, 0.04910428440611303, 0....",b,0.136846,0.251902
8503,"[0.06963166281050472, 0.010390805899488929, 0....",b,0.161627,0.251905
1902,"[0.052072044971607355, 0.08555690921794881, 0....",b,0.132761,0.251909
3814,"[0.06882985380447625, 0.06334309029378724, 0.0...",b,0.172988,0.251916


In [8]:
df_portafolios[df_portafolios['Clasificación']=='m'].sort_values('Volatilidad')

,Portafolio,Clasificación,Rendimiento,Volatilidad
2558,"[0.06985931991466496, 0.026188527857782595, 0....",m,0.161545,0.251921
8064,"[0.0761945139682161, 0.09077972114542616, 0.00...",m,0.179644,0.251929
1271,"[0.09709180030306469, 0.029671383822420398, 0....",m,0.165265,0.251938
9470,"[0.07322283240061737, 0.044197138651264345, 0....",m,0.144857,0.251948
5924,"[0.05118714075533857, 0.06620946759802318, 0.0...",m,0.148959,0.251959
...,...,...,...,...
2312,"[0.021319017375068798, 0.05709588599025497, 0....",m,0.168476,0.262594
1688,"[0.06401988480153276, 0.03849070612856018, 0.0...",m,0.164648,0.262597
9430,"[1.3105669578278979e-05, 0.05479241881611664, ...",m,0.135020,0.262599
2867,"[0.08226646741224314, 0.01727136695934198, 0.0...",m,0.184661,0.262601


In [9]:
df_portafolios[df_portafolios['Clasificación']=='a'].sort_values('Volatilidad')

,Portafolio,Clasificación,Rendimiento,Volatilidad
8235,"[0.07464685127850228, 0.04184794606766372, 0.0...",a,0.158927,0.262610
9368,"[0.07228952528566997, 0.006334245673154502, 0....",a,0.194263,0.262618
7260,"[0.005096025350213679, 0.009600973702052578, 0...",a,0.179287,0.262634
1389,"[0.09293569769450485, 0.01603760265996428, 0.0...",a,0.164590,0.262636
7399,"[0.027543353337310773, 0.05651622226615879, 0....",a,0.149433,0.262643
...,...,...,...,...
484,"[0.04427873864667153, 0.04039127094200349, 0.0...",a,0.238313,0.313885
5831,"[0.05520994212212064, 0.02021163396847485, 0.0...",a,0.238693,0.314493
4172,"[0.025745973729639528, 0.0942566463155192, 0.0...",a,0.182984,0.318691
3041,"[0.06803661270585266, 0.06537048533363746, 0.0...",a,0.194230,0.318908
